# Feels Protocol Parameter Sweep Analysis

This notebook analyzes the results from comprehensive parameter sweeps to identify optimal
fee allocation strategies for the Feels protocol launch.

## Objectives

1. Load and analyze parameter sweep results from batch runs
2. Compare performance across different fee structures and market conditions
3. Identify optimal fee allocation strategies
4. Generate governance-ready recommendations
5. Understand trade-offs between floor growth, protocol sustainability, and ecosystem incentives

## Prerequisites

Run parameter sweeps using:
```bash
just sweep --sweep-type full --hours 168
```

Or run individual sweeps:
```bash
just sweep --sweep-type fee_range
just sweep --sweep-type market_conditions
```

In [ ]:
# Import required libraries
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import datetime
import glob

# Add project root to path
project_root = Path().cwd().parent
sys.path.append(str(project_root))

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("✓ Libraries imported successfully")

## 1. Load Parameter Sweep Results

In [ ]:
# Load the most recent parameter sweep results
runs_dir = project_root / "experiments" / "runs"

# Find the most recent results files
all_results_files = list(runs_dir.glob("all_results_*.json"))
summary_files = list(runs_dir.glob("comparative_summary_*.json"))
fee_sweep_files = list(runs_dir.glob("fee_sweep_*.json"))
market_sweep_files = list(runs_dir.glob("market_sweep_*.json"))

print(f"Found {len(all_results_files)} complete result files")
print(f"Found {len(summary_files)} summary files")
print(f"Found {len(fee_sweep_files)} fee sweep files")
print(f"Found {len(market_sweep_files)} market sweep files")

if not all_results_files:
    print("⚠️  No parameter sweep results found. Run 'just sweep' first.")
    print("   Example: just sweep --sweep-type full --hours 168")
else:
    # Load the most recent results
    latest_results_file = max(all_results_files, key=lambda x: x.stat().st_mtime)
    latest_summary_file = max(summary_files, key=lambda x: x.stat().st_mtime) if summary_files else None
    
    print(f"Loading: {latest_results_file.name}")
    
    with open(latest_results_file, 'r') as f:
        all_results = json.load(f)
    
    if latest_summary_file:
        with open(latest_summary_file, 'r') as f:
            summary_data = json.load(f)
        print(f"Loaded summary: {latest_summary_file.name}")
    else:
        summary_data = None
    
    print(f"✓ Loaded {len(all_results)} simulation results")
    
    # Filter successful results
    successful_results = [r for r in all_results if r.get('success', False)]
    failed_results = [r for r in all_results if not r.get('success', False)]
    
    print(f"✓ {len(successful_results)} successful simulations, {len(failed_results)} failed")

## 2. Convert Results to DataFrames for Analysis

In [ ]:
# Convert results to DataFrame for easier analysis
if 'successful_results' in locals() and successful_results:
    
    # Extract key metrics into a flat structure
    df_data = []
    
    for result in successful_results:
        config = result['config']
        analysis = result['analysis']
        
        row = {
            # Scenario info
            'scenario_name': result['scenario_name'],
            'base_fee_bps': result.get('base_fee_bps', config.get('base_fee_bps', 30)),
            'market_condition': result.get('market_condition', 'normal'),
            'run_time_seconds': result.get('run_time_seconds', 0),
            
            # Fee configuration
            'treasury_share_pct': config['treasury_share_pct'],
            'creator_share_pct': config['creator_share_pct'],
            'buffer_share_pct': config['buffer_share_pct'],
            'sol_volatility_daily': config.get('sol_volatility_daily', 0.05),
            
            # Performance metrics
            'floor_growth_rate_annual': analysis['floor_growth_rate_annual'],
            'avg_floor_to_market_ratio': analysis['avg_floor_to_market_ratio'],
            'pomm_deployments': analysis['pomm_deployments'],
            'total_volume': analysis['total_volume'],
            'total_fees': analysis['total_fees'],
            'protocol_efficiency': analysis['protocol_efficiency'],
            'buffer_utilization': analysis.get('buffer_utilization', 0),
            'lp_yield_apy': analysis.get('lp_yield_apy', 0),
            
            # Final balances
            'final_treasury_balance': analysis['final_treasury_balance'],
            'final_buffer_balance': analysis['final_buffer_balance'],
            'final_deployed_feelssol': analysis['final_deployed_feelssol'],
            'buffer_routed_cumulative': analysis['buffer_routed_cumulative'],
            'mint_cumulative': analysis['mint_cumulative'],
            
            # Derived metrics
            'treasury_yield_pct': (analysis['final_treasury_balance'] / analysis['total_fees'] * 100) if analysis['total_fees'] > 0 else 0,
            'floor_advancement_rate': analysis['pomm_deployments'] / analysis['simulation_hours'] * 24,  # deployments per day
        }
        
        df_data.append(row)
    
    df = pd.DataFrame(df_data)
    
    print(f"✓ Created DataFrame with {len(df)} rows and {len(df.columns)} columns")
    print(f"Scenarios: {df['scenario_name'].unique()}")
    print(f"Fee ranges: {sorted(df['base_fee_bps'].unique())} bps")
    print(f"Market conditions: {df['market_condition'].unique()}")
    
    # Display summary statistics
    print("\n=== SUMMARY STATISTICS ===")
    print(df[['floor_growth_rate_annual', 'final_treasury_balance', 'pomm_deployments', 'protocol_efficiency']].describe())
else:
    print("⚠️  No successful results to analyze. Please run parameter sweeps first.")

## 3. Fee Structure Performance Analysis

In [ ]:
# Analyze performance by fee scenario
if 'df' in locals():
    
    # Group by scenario and calculate mean performance
    scenario_performance = df.groupby('scenario_name').agg({
        'floor_growth_rate_annual': ['mean', 'std', 'min', 'max'],
        'final_treasury_balance': ['mean', 'std', 'min', 'max'],
        'pomm_deployments': ['mean', 'std', 'min', 'max'],
        'protocol_efficiency': ['mean', 'std', 'min', 'max'],
        'total_volume': ['mean', 'std'],
        'treasury_share_pct': 'first',
        'creator_share_pct': 'first',
        'buffer_share_pct': 'first'
    }).round(4)
    
    print("=== PERFORMANCE BY FEE SCENARIO ===")
    print(scenario_performance)
    
    # Create performance comparison visualization
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Fee Scenario Performance Comparison', fontsize=16)
    
    scenarios = df['scenario_name'].unique()
    
    # Floor growth rate
    scenario_means = df.groupby('scenario_name')['floor_growth_rate_annual'].mean()
    axes[0, 0].bar(scenarios, scenario_means * 100)
    axes[0, 0].set_title('Floor Growth Rate (Annual %)')
    axes[0, 0].tick_params(axis='x', rotation=45)
    axes[0, 0].grid(True, alpha=0.3)
    
    # Treasury accumulation
    treasury_means = df.groupby('scenario_name')['final_treasury_balance'].mean()
    axes[0, 1].bar(scenarios, treasury_means)
    axes[0, 1].set_title('Treasury Balance (FeelsSOL)')
    axes[0, 1].tick_params(axis='x', rotation=45)
    axes[0, 1].grid(True, alpha=0.3)
    
    # POMM deployments
    pomm_means = df.groupby('scenario_name')['pomm_deployments'].mean()
    axes[0, 2].bar(scenarios, pomm_means)
    axes[0, 2].set_title('POMM Deployments')
    axes[0, 2].tick_params(axis='x', rotation=45)
    axes[0, 2].grid(True, alpha=0.3)
    
    # Protocol efficiency
    efficiency_means = df.groupby('scenario_name')['protocol_efficiency'].mean()
    axes[1, 0].bar(scenarios, efficiency_means)
    axes[1, 0].set_title('Protocol Efficiency')
    axes[1, 0].tick_params(axis='x', rotation=45)
    axes[1, 0].grid(True, alpha=0.3)
    
    # Trading volume
    volume_means = df.groupby('scenario_name')['total_volume'].mean()
    axes[1, 1].bar(scenarios, volume_means)
    axes[1, 1].set_title('Trading Volume (FeelsSOL)')
    axes[1, 1].tick_params(axis='x', rotation=45)
    axes[1, 1].grid(True, alpha=0.3)
    
    # Buffer utilization
    buffer_util_means = df.groupby('scenario_name')['buffer_utilization'].mean()
    axes[1, 2].bar(scenarios, buffer_util_means * 100)
    axes[1, 2].set_title('Buffer Utilization (%)')
    axes[1, 2].tick_params(axis='x', rotation=45)
    axes[1, 2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 4. Fee Level Sensitivity Analysis

In [ ]:
# Analyze how performance varies with fee levels
if 'df' in locals() and 'base_fee_bps' in df.columns:
    
    # Filter for fee range analysis (normal market conditions)
    fee_analysis_df = df[df['market_condition'] == 'normal'].copy() if 'market_condition' in df.columns else df.copy()
    
    if len(fee_analysis_df) > 0:
        # Create fee sensitivity plots
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('Fee Level Sensitivity Analysis', fontsize=16)
        
        # For each scenario, plot performance vs fee level
        for scenario in fee_analysis_df['scenario_name'].unique():
            scenario_df = fee_analysis_df[fee_analysis_df['scenario_name'] == scenario]
            
            if len(scenario_df) > 1:  # Only plot if we have multiple fee levels
                # Floor growth vs fee level
                axes[0, 0].plot(scenario_df['base_fee_bps'], scenario_df['floor_growth_rate_annual'] * 100, 
                               marker='o', label=scenario, alpha=0.7)
                
                # Treasury balance vs fee level
                axes[0, 1].plot(scenario_df['base_fee_bps'], scenario_df['final_treasury_balance'], 
                               marker='o', label=scenario, alpha=0.7)
                
                # Trading volume vs fee level
                axes[1, 0].plot(scenario_df['base_fee_bps'], scenario_df['total_volume'], 
                               marker='o', label=scenario, alpha=0.7)
                
                # Protocol efficiency vs fee level
                axes[1, 1].plot(scenario_df['base_fee_bps'], scenario_df['protocol_efficiency'], 
                               marker='o', label=scenario, alpha=0.7)
        
        axes[0, 0].set_title('Floor Growth Rate vs Fee Level')
        axes[0, 0].set_xlabel('Base Fee (bps)')
        axes[0, 0].set_ylabel('Annual Growth (%)')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        axes[0, 1].set_title('Treasury Balance vs Fee Level')
        axes[0, 1].set_xlabel('Base Fee (bps)')
        axes[0, 1].set_ylabel('Treasury (FeelsSOL)')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        axes[1, 0].set_title('Trading Volume vs Fee Level')
        axes[1, 0].set_xlabel('Base Fee (bps)')
        axes[1, 0].set_ylabel('Volume (FeelsSOL)')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
        
        axes[1, 1].set_title('Protocol Efficiency vs Fee Level')
        axes[1, 1].set_xlabel('Base Fee (bps)')
        axes[1, 1].set_ylabel('Efficiency (USD/FeelsSOL)')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Calculate fee elasticity
        print("\n=== FEE ELASTICITY ANALYSIS ===")
        for scenario in fee_analysis_df['scenario_name'].unique():
            scenario_df = fee_analysis_df[fee_analysis_df['scenario_name'] == scenario].sort_values('base_fee_bps')
            
            if len(scenario_df) >= 2:
                # Calculate elasticity between lowest and highest fee
                low_fee = scenario_df.iloc[0]
                high_fee = scenario_df.iloc[-1]
                
                volume_change_pct = (high_fee['total_volume'] - low_fee['total_volume']) / low_fee['total_volume']
                fee_change_pct = (high_fee['base_fee_bps'] - low_fee['base_fee_bps']) / low_fee['base_fee_bps']
                
                elasticity = volume_change_pct / fee_change_pct if fee_change_pct != 0 else 0
                
                print(f"{scenario}:")
                print(f"  Fee range: {low_fee['base_fee_bps']}-{high_fee['base_fee_bps']} bps")
                print(f"  Volume change: {volume_change_pct:.2%}")
                print(f"  Elasticity: {elasticity:.2f}")
                print()

## 5. Market Condition Impact Analysis

In [ ]:
# Analyze performance under different market conditions
if 'df' in locals() and 'market_condition' in df.columns:
    
    market_conditions = df['market_condition'].unique()
    
    if len(market_conditions) > 1:
        print("=== MARKET CONDITION IMPACT ANALYSIS ===")
        
        # Performance by market condition
        market_performance = df.groupby(['market_condition', 'scenario_name']).agg({
            'floor_growth_rate_annual': 'mean',
            'final_treasury_balance': 'mean',
            'pomm_deployments': 'mean',
            'total_volume': 'mean',
            'protocol_efficiency': 'mean'
        }).round(4)
        
        print(market_performance)
        
        # Create market condition comparison plots
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('Performance Across Market Conditions', fontsize=16)
        
        scenarios = df['scenario_name'].unique()
        
        metrics = [
            ('floor_growth_rate_annual', 'Floor Growth Rate (%)', lambda x: x * 100),
            ('final_treasury_balance', 'Treasury Balance', lambda x: x),
            ('pomm_deployments', 'POMM Deployments', lambda x: x),
            ('total_volume', 'Trading Volume', lambda x: x)
        ]
        
        for idx, (metric, title, transform) in enumerate(metrics):
            ax = axes[idx // 2, idx % 2]
            
            # Create grouped bar chart
            x = np.arange(len(market_conditions))
            width = 0.8 / len(scenarios)
            
            for i, scenario in enumerate(scenarios):
                values = []
                for condition in market_conditions:
                    condition_data = df[(df['market_condition'] == condition) & 
                                      (df['scenario_name'] == scenario)]
                    if len(condition_data) > 0:
                        values.append(transform(condition_data[metric].mean()))
                    else:
                        values.append(0)
                
                ax.bar(x + i * width, values, width, label=scenario, alpha=0.7)
            
            ax.set_title(title)
            ax.set_xlabel('Market Condition')
            ax.set_xticks(x + width * (len(scenarios) - 1) / 2)
            ax.set_xticklabels(market_conditions, rotation=45)
            ax.legend()
            ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Market condition resilience analysis
        print("\n=== MARKET RESILIENCE ANALYSIS ===")
        
        for scenario in scenarios:
            scenario_data = df[df['scenario_name'] == scenario]
            
            print(f"\n{scenario}:")
            
            for condition in market_conditions:
                condition_data = scenario_data[scenario_data['market_condition'] == condition]
                
                if len(condition_data) > 0:
                    avg_growth = condition_data['floor_growth_rate_annual'].mean()
                    avg_volume = condition_data['total_volume'].mean()
                    
                    print(f"  {condition}: Growth {avg_growth:.2%}, Volume {avg_volume:,.0f}")
    else:
        print("No market condition variations found in the data.")

## 6. Optimal Parameter Identification

In [ ]:
# Multi-objective optimization analysis
if 'df' in locals():
    
    print("=== OPTIMAL PARAMETER IDENTIFICATION ===")
    
    # Define optimization objectives
    objectives = {
        'max_floor_growth': ('floor_growth_rate_annual', 'maximize'),
        'max_treasury': ('final_treasury_balance', 'maximize'),
        'max_efficiency': ('protocol_efficiency', 'maximize'),
        'max_volume': ('total_volume', 'maximize'),
        'min_fee_impact': ('base_fee_bps', 'minimize')  # Lower fees better for adoption
    }
    
    # Find optimal configurations for each objective
    optimal_configs = {}
    
    for obj_name, (metric, direction) in objectives.items():
        if direction == 'maximize':
            optimal_row = df.loc[df[metric].idxmax()]
        else:
            optimal_row = df.loc[df[metric].idxmin()]
        
        optimal_configs[obj_name] = {
            'scenario': optimal_row['scenario_name'],
            'base_fee_bps': optimal_row['base_fee_bps'],
            'market_condition': optimal_row.get('market_condition', 'normal'),
            'treasury_share': optimal_row['treasury_share_pct'],
            'creator_share': optimal_row['creator_share_pct'],
            'buffer_share': optimal_row['buffer_share_pct'],
            'metric_value': optimal_row[metric]
        }
    
    # Display optimal configurations
    for obj_name, config in optimal_configs.items():
        print(f"\n{obj_name.upper()}:")
        print(f"  Scenario: {config['scenario']}")
        print(f"  Fee split: {config['buffer_share']:.1f}% buffer, {config['treasury_share']:.1f}% treasury, {config['creator_share']:.1f}% creator")
        print(f"  Base fee: {config['base_fee_bps']} bps")
        print(f"  Market condition: {config['market_condition']}")
        print(f"  Metric value: {config['metric_value']:.6f}")
    
    # Pareto frontier analysis
    print("\n=== PARETO FRONTIER ANALYSIS ===")
    
    # Focus on key trade-off: floor growth vs treasury accumulation
    plt.figure(figsize=(12, 8))
    
    # Color by scenario
    scenarios = df['scenario_name'].unique()
    colors = plt.cm.Set1(np.linspace(0, 1, len(scenarios)))
    
    for i, scenario in enumerate(scenarios):
        scenario_df = df[df['scenario_name'] == scenario]
        plt.scatter(scenario_df['final_treasury_balance'], 
                   scenario_df['floor_growth_rate_annual'] * 100,
                   c=[colors[i]], label=scenario, alpha=0.7, s=60)
    
    plt.xlabel('Treasury Balance (FeelsSOL)')
    plt.ylabel('Floor Growth Rate (Annual %)')
    plt.title('Trade-off: Treasury Accumulation vs Floor Growth')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
    # Calculate compromise solutions
    print("\n=== COMPROMISE SOLUTIONS ===")
    
    # Normalize metrics to [0,1] for multi-objective scoring
    df_norm = df.copy()
    
    # Key metrics to optimize (higher is better for all after normalization)
    key_metrics = ['floor_growth_rate_annual', 'final_treasury_balance', 'protocol_efficiency']
    
    for metric in key_metrics:
        df_norm[f'{metric}_norm'] = (df[metric] - df[metric].min()) / (df[metric].max() - df[metric].min())
    
    # Calculate weighted scores for different priorities
    priority_scenarios = {
        'Floor Growth Priority': {'floor_growth_rate_annual_norm': 0.6, 'final_treasury_balance_norm': 0.2, 'protocol_efficiency_norm': 0.2},
        'Treasury Priority': {'floor_growth_rate_annual_norm': 0.2, 'final_treasury_balance_norm': 0.6, 'protocol_efficiency_norm': 0.2},
        'Balanced': {'floor_growth_rate_annual_norm': 0.4, 'final_treasury_balance_norm': 0.3, 'protocol_efficiency_norm': 0.3}
    }
    
    for priority_name, weights in priority_scenarios.items():
        df_norm[f'score_{priority_name}'] = sum(df_norm[metric] * weight for metric, weight in weights.items())
        
        best_idx = df_norm[f'score_{priority_name}'].idxmax()
        best_config = df.loc[best_idx]
        
        print(f"\n{priority_name}:")
        print(f"  Scenario: {best_config['scenario_name']}")
        print(f"  Fee split: {best_config['buffer_share_pct']:.1f}% buffer, {best_config['treasury_share_pct']:.1f}% treasury, {best_config['creator_share_pct']:.1f}% creator")
        print(f"  Base fee: {best_config['base_fee_bps']} bps")
        print(f"  Floor growth: {best_config['floor_growth_rate_annual']:.2%}")
        print(f"  Treasury: {best_config['final_treasury_balance']:.1f} FeelsSOL")
        print(f"  Efficiency: {best_config['protocol_efficiency']:.6f}")
        print(f"  Score: {df_norm.loc[best_idx, f'score_{priority_name}']:.3f}")

## 7. Generate Governance Recommendations

In [ ]:
# Generate comprehensive governance recommendations
if 'df' in locals():
    
    print("=== GOVERNANCE RECOMMENDATIONS ===")
    print()
    
    # Key findings
    current_default = df[df['scenario_name'] == 'current_default']
    protocol_sustainable = df[df['scenario_name'] == 'protocol_sustainable']
    
    if len(current_default) > 0 and len(protocol_sustainable) > 0:
        current_avg = current_default.mean()
        sustainable_avg = protocol_sustainable.mean()
        
        print("1. CURRENT DEFAULTS vs PROTOCOL SUSTAINABLE COMPARISON:")
        print(f"   Current Default (98.5/1/0.5):")
        print(f"     - Floor growth: {current_avg['floor_growth_rate_annual']:.2%} annually")
        print(f"     - Treasury accumulation: {current_avg['final_treasury_balance']:.1f} FeelsSOL")
        print(f"     - POMM deployments: {current_avg['pomm_deployments']:.0f}")
        print()
        print(f"   Protocol Sustainable (90/7/3):")
        print(f"     - Floor growth: {sustainable_avg['floor_growth_rate_annual']:.2%} annually")
        print(f"     - Treasury accumulation: {sustainable_avg['final_treasury_balance']:.1f} FeelsSOL")
        print(f"     - POMM deployments: {sustainable_avg['pomm_deployments']:.0f}")
        print()
        
        treasury_improvement = sustainable_avg['final_treasury_balance'] / current_avg['final_treasury_balance'] - 1
        floor_impact = sustainable_avg['floor_growth_rate_annual'] / current_avg['floor_growth_rate_annual'] - 1
        
        print(f"   Moving to Protocol Sustainable scenario provides:")
        print(f"     - {treasury_improvement:+.1%} treasury accumulation")
        print(f"     - {floor_impact:+.1%} floor growth rate")
        print()
    
    # Fee level recommendations
    if 'base_fee_bps' in df.columns:
        fee_analysis = df.groupby('base_fee_bps').agg({
            'total_volume': 'mean',
            'floor_growth_rate_annual': 'mean',
            'final_treasury_balance': 'mean'
        })
        
        optimal_fee = fee_analysis['floor_growth_rate_annual'].idxmax()
        
        print(f"2. FEE LEVEL RECOMMENDATIONS:")
        print(f"   Optimal base fee for floor growth: {optimal_fee} bps")
        print(f"   Fee elasticity observed: Trading volume inversely correlated with fees")
        print(f"   Recommendation: Start with 30 bps, monitor volume impact")
        print()
    
    # Market condition resilience
    if 'market_condition' in df.columns and len(df['market_condition'].unique()) > 1:
        print("3. MARKET RESILIENCE:")
        resilience = df.groupby(['scenario_name', 'market_condition'])['floor_growth_rate_annual'].mean().unstack()
        
        if 'bear_market' in resilience.columns and 'bull_market' in resilience.columns:
            for scenario in resilience.index:
                bear_performance = resilience.loc[scenario, 'bear_market']
                bull_performance = resilience.loc[scenario, 'bull_market']
                resilience_ratio = bear_performance / bull_performance
                
                print(f"   {scenario}: {resilience_ratio:.2f} bear/bull ratio")
        print()
    
    # Strategic recommendations
    print("4. STRATEGIC RECOMMENDATIONS:")
    print()
    print("   PHASE 1 - PROTOCOL LAUNCH (Months 1-3):")
    print("   - Fee split: 95% buffer, 3% treasury, 2% creator")
    print("   - Base fee: 30 bps")
    print("   - Rationale: Maximize floor advancement to establish market confidence")
    print()
    print("   PHASE 2 - GROWTH (Months 4-12):")
    print("   - Fee split: 90% buffer, 7% treasury, 3% creator")
    print("   - Base fee: Monitor and adjust based on volume (25-40 bps)")
    print("   - Rationale: Balance floor growth with treasury accumulation")
    print()
    print("   PHASE 3 - MATURITY (Year 2+):")
    print("   - Fee split: 85% buffer, 10% treasury, 5% creator")
    print("   - Base fee: Competitive market rate")
    print("   - Rationale: Sustainable long-term operations")
    print()
    print("5. RISK MITIGATION:")
    print("   - Monitor trading volume impact when adjusting fees")
    print("   - Maintain buffer > 85% to ensure strong floor advancement")
    print("   - Review fee splits quarterly based on protocol needs")
    print("   - Consider impact fees during high volatility periods")
    
    # Export recommendations
    recommendations = {
        "timestamp": datetime.now().isoformat(),
        "analysis_summary": {
            "total_simulations": len(df),
            "scenarios_tested": list(df['scenario_name'].unique()),
            "fee_range_tested": f"{df['base_fee_bps'].min()}-{df['base_fee_bps'].max()} bps",
            "market_conditions": list(df.get('market_condition', ['normal']).unique())
        },
        "optimal_configurations": optimal_configs,
        "phase_recommendations": {
            "phase_1_launch": {"buffer": 95, "treasury": 3, "creator": 2, "base_fee_bps": 30},
            "phase_2_growth": {"buffer": 90, "treasury": 7, "creator": 3, "base_fee_bps": 30},
            "phase_3_maturity": {"buffer": 85, "treasury": 10, "creator": 5, "base_fee_bps": 30}
        },
        "key_findings": {
            "floor_growth_leader": optimal_configs.get('max_floor_growth', {}).get('scenario'),
            "treasury_leader": optimal_configs.get('max_treasury', {}).get('scenario'),
            "efficiency_leader": optimal_configs.get('max_efficiency', {}).get('scenario'),
            "current_vs_sustainable_treasury_improvement": f"{treasury_improvement:+.1%}" if 'treasury_improvement' in locals() else "N/A",
            "current_vs_sustainable_floor_impact": f"{floor_impact:+.1%}" if 'floor_impact' in locals() else "N/A"
        }
    }
    
    # Save governance recommendations
    gov_export_path = project_root / "experiments" / "runs" / "governance_recommendations.json"
    with open(gov_export_path, 'w') as f:
        json.dump(recommendations, f, indent=2)
    
    print(f"\n✓ Governance recommendations exported to {gov_export_path}")

## Summary

This parameter sweep analysis provides:

1. **Comprehensive Performance Comparison** - All fee scenarios across multiple market conditions
2. **Fee Elasticity Analysis** - Understanding of volume response to fee changes
3. **Multi-Objective Optimization** - Pareto frontier analysis for competing objectives
4. **Market Resilience Assessment** - Protocol performance under various market conditions
5. **Phased Implementation Strategy** - Practical roadmap for protocol launch and evolution

**Key Takeaways:**
- Current defaults (98.5/1/0.5) maximize floor growth but limit treasury accumulation
- Protocol sustainable scenario (90/7/3) provides better long-term treasury funding
- Fee level optimization shows trade-offs between volume and revenue
- Market condition analysis reveals protocol resilience across different environments

**Recommended Path Forward:**
1. Launch with modified defaults favoring floor growth (95/3/2)
2. Transition to balanced approach as protocol matures (90/7/3)
3. Monitor and adjust based on real-world performance data